In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
import numpy as np
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
img_gen = ImageDataGenerator(rescale= 1/255)

In [ ]:
train_data = img_gen.flow_from_directory("/content/gdrive/MyDrive/dataset/top50_full_stickman/train/", 
                                         class_mode='categorical', 
                                         batch_size=16, 
                                         target_size=(224, 224),
                                         seed=123
                                         )

Found 45015 images belonging to 50 classes.


In [ ]:
valid_data = img_gen.flow_from_directory("/content/gdrive/MyDrive/dataset/top50_full_stickman/val/", 
                                         class_mode='categorical',  
                                         batch_size=16, 
                                         target_size=(224, 224),
                                         seed=123
                                         )

Found 3531 images belonging to 50 classes.


In [ ]:
test_data = img_gen.flow_from_directory("/content/gdrive/MyDrive/dataset/top50_full_stickman/test/", 
                                         class_mode='categorical',  
                                         batch_size=1, 
                                         target_size=(224, 224),
                                         shuffle = False,  #by default shuffle = True. shuffle = False: sorts the data in alphanumeric order.
                                         seed=123
                                         )   

Found 5082 images belonging to 50 classes.


In [ ]:
test_data.labels

array([ 0,  0,  0, ..., 49, 49, 49], dtype=int32)

In [ ]:
icp = InceptionV3(
            input_shape=(224, 224, 3),  
            include_top=False,   
            weights='imagenet' 
            )

for layer in icp.layers:
    layer.trainable = True 

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
x = Flatten()(icp.output)
x = Dense(128, activation = 'relu', name='fc1')(x)  #tried hidden units = 4096, test_accuracy = 0.6
x = Dense(128, activation = 'relu', name='fc2')(x)
x = Dense(50, activation='softmax', name='predictions')(x)

model = Model(inputs = icp.input, outputs = x)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import TopKCategoricalAccuracy
model.compile(optimizer=Adam(learning_rate=0.000002), loss='categorical_crossentropy', metrics=['accuracy', 
                                                                                                    #'sparse_categorical_accuracy', 
                                                                                                    TopKCategoricalAccuracy(k=3),
                                                                                                    #TopKCategoricalAccuracy(k=5),
                                                                                                    ])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(train_data, 
          epochs=20, 
          # batch_size=16, 
          validation_data=valid_data, 
          callbacks=[callback],
          steps_per_epoch=len(train_data), 
          validation_steps=len(valid_data)
          )

Epoch 1/20
2814/2814 [==============================] - 8921s 3s/step - loss: 3.3159 - accuracy: 0.1817 - top_k_categorical_accuracy: 0.3260 - val_loss: 3.2465 - val_accuracy: 0.1677 - val_top_k_categorical_accuracy: 0.3381
Epoch 2/20
2814/2814 [==============================] - 395s 140ms/step - loss: 2.1069 - accuracy: 0.4957 - top_k_categorical_accuracy: 0.6723 - val_loss: 2.9372 - val_accuracy: 0.2583 - val_top_k_categorical_accuracy: 0.4642
Epoch 3/20
2814/2814 [==============================] - 402s 143ms/step - loss: 1.3841 - accuracy: 0.6746 - top_k_categorical_accuracy: 0.8153 - val_loss: 2.7869 - val_accuracy: 0.3161 - val_top_k_categorical_accuracy: 0.5217
Epoch 4/20
2814/2814 [==============================] - 402s 143ms/step - loss: 0.9663 - accuracy: 0.7722 - top_k_categorical_accuracy: 0.8826 - val_loss: 2.7835 - val_accuracy: 0.3277 - val_top_k_categorical_accuracy: 0.5463
Epoch 5/20
2814/2814 [==============================] - 399s 142ms/step - loss: 0.7007 - accuracy:

In [ ]:
import pickle
filename = '/content/gdrive/MyDrive/dataset/inception_stickman_50.pkl'
outfile = open(filename,'wb')

pickle.dump(model,outfile)
outfile.close()

In [ ]:
result = model.evaluate(test_data)

5082/5082 [==============================] - 1554s 306ms/step - loss: 2.7665 - accuracy: 0.3650 - top_k_categorical_accuracy: 0.5368


In [ ]:
result

[2.7665059566497803, 0.3650137782096863, 0.536796510219574]